Ejercicio: Sintonice el área bajo la curva

En este ejercicio, crearemos y compararemos dos modelos usando curvas ROC y ajustaremos uno usando el área bajo la curva (AUC).

El objetivo de nuestros modelos es identificar si cada elemento detectado en la montaña es un caminante (verdadero) o un árbol (falso). Trabajaremos con nuestra función de movimiento aquí. Vamos a ver:

In [ ]:
import numpy
import pandas
!pip install statsmodels
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/graphing.py
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/Data/hiker_or_tree.csv
!wget https://raw.githubusercontent.com/MicrosoftDocs/mslearn-introduction-to-machine-learning/main/m2d_make_roc.py
import graphing # custom graphing code. See our GitHub repo for details
import sklearn.model_selection

# Load our data from disk
df = pandas.read_csv("hiker_or_tree.csv", delimiter="\\t")

# Remove features we no longer want
del df["height"]
del df["texture"]

# Split into train and test
train, test =  sklearn.model_selection.train_test_split(df, test_size=0.5, random_state=1)

# Graph our feature
graphing.multiple_histogram(test, label_x="motion", label_group="is_hiker", nbins=12)

El movimiento parece estar más asociado con los excursionistas que con los árboles, pero no perfectamente. Presumiblemente, esto se debe a que los árboles se mueven con el viento y algunos excursionistas se encuentran sentados.

Un modelo de regresión logística y un bosque aleatorio
Entrenemos el mismo modelo de regresión logística que usamos en el ejercicio anterior, así como un modelo de bosque aleatorio. Ambos intentarán predecir qué objetos son excursionistas.

Primero la regresión logística:

In [ ]:
import statsmodels.api
from sklearn.metrics import accuracy_score

# This is a helper method that reformats the data to be compatible
# with this particular logistic regression model 
prep_data = lambda x:  numpy.column_stack((numpy.full(x.shape, 1), x))

# Train a logistic regression model to predict hiker based on motion
lr_model = statsmodels.api.Logit(train.is_hiker, prep_data(train.motion), add_constant=True).fit()

# Assess its performance
# -- Train
predictions = lr_model.predict(prep_data(train.motion)) > 0.5
train_accuracy = accuracy_score(train.is_hiker, predictions)

# -- Test
predictions = lr_model.predict(prep_data(test.motion)) > 0.5
test_accuracy = accuracy_score(test.is_hiker, predictions)

print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)

# Plot the model
predict_with_logistic_regression = lambda x: lr_model.predict(prep_data(x))
graphing.scatter_2D(test, label_x="motion", label_y="is_hiker", title="Logistic Regression", trendline=predict_with_logistic_regression)

Ahora nuestro modelo de bosque aleatorio:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Create a random forest model with 50 trees
random_forest = RandomForestClassifier(random_state=2,
                                       verbose=False)

# Train the model
random_forest.fit(train[["motion"]], train.is_hiker)

# Assess its performance
# -- Train
predictions = random_forest.predict(train[["motion"]])
train_accuracy = accuracy_score(train.is_hiker, predictions)

# -- Test
predictions = random_forest.predict(test[["motion"]])
test_accuracy = accuracy_score(test.is_hiker, predictions)


# Train and test the model
print("Random Forest Performance:")
print("Train accuracy", train_accuracy)
print("Test accuracy", test_accuracy)


Estos modelos tienen un rendimiento similar, pero no idéntico, en el conjunto de prueba en términos de precisión.

Crear gráficos ROC
Vamos a crear curvas ROC para estos modelos. Para ello, simplemente importaremos el código de los últimos ejercicios para poder centrarnos en lo que nos gustaría aprender aquí. Si necesita un repaso sobre cómo se hicieron, vuelva a leer el último ejercicio.

Tenga en cuenta que hemos hecho un ligero cambio. Ahora nuestro método produce tanto un gráfico como la tabla de números que usamos para crear el gráfico.

Primero veamos el modelo de regresión logística:

In [ ]:
from m2d_make_roc import create_roc_curve # import our previous ROC code

fig, thresholds_lr = create_roc_curve(predict_with_logistic_regression, test, "motion")

# Uncomment the line below if you would like to see the area under the curve
#fig.update_traces(fill="tozeroy")

fig.show()

# Show the table of results
thresholds_lr

Podemos ver que nuestro modelo funciona mejor que el azar (no es una línea diagonal). Nuestra tabla muestra la tasa de falsos positivos (fpr) y la tasa de verdaderos positivos (tpr) para cada umbral.

Repitamos esto para nuestro modelo de bosque aleatorio:

In [ ]:
# Don't worry about this lambda function. It simply reorganizes 
# the data into the shape expected by the random forest model, 
# and calls predict_proba, which gives us predicted probabilities
# that the label is 'hiker'
predict_with_random_forest = lambda x: random_forest.predict_proba(numpy.array(x).reshape(-1, 1))[:,1]

# Create the ROC curve
fig, thresholds_rf = create_roc_curve(predict_with_random_forest, test, "motion")

# Uncomment the line below if you would like to see the area under the curve
#fig.update_traces(fill="tozeroy")

fig.show()

# Show the table of results
thresholds_lr

Área bajo la curva

Nuestros modelos se parecen bastante. ¿Qué modelo creemos que es mejor? Usemos el área bajo la curva (AUC) para compararlos. Deberíamos esperar un número mayor que 0,5, porque ambos modelos son mejores que el azar, pero menor que 1, porque no son perfectos.

In [ ]:
from sklearn.metrics import roc_auc_score

# Logistic regression
print("Logistic Regression AUC:", roc_auc_score(test.is_hiker, predict_with_logistic_regression(test.motion)))

# Random Forest
print("Random Forest AUC:", roc_auc_score(test.is_hiker, predict_with_random_forest(test.motion)))

Por un margen muy pequeño, el modelo de regresión logística se destaca.

Recuerde, esto no significa que el modelo de regresión logística siempre funcionará mejor que el bosque aleatorio. Significa que el modelo de regresión logística es una opción ligeramente mejor para este tipo de datos, y probablemente depende un poco menos de tener los umbrales de decisión perfectos elegidos.

Ajuste del umbral de decisión
También podemos usar nuestra información de ROC para encontrar los mejores umbrales para usar. Solo trabajaremos con nuestro modelo de bosque aleatorio para esta parte.

Primero, echemos un vistazo a la tasa de verdaderos y falsos positivos con el umbral predeterminado de 0,5:

In [ ]:
# Print out its expected performance at the default threshold of 0.5
# We previously obtained this information when we created our graphs
row_of_0point5 = thresholds_rf[thresholds_rf.threshold == 0.5]
print("TPR at threshold of 0.5:", row_of_0point5.tpr.values[0])
print("FPR at threshold of 0.5:", row_of_0point5.fpr.values[0])

Podemos esperar que, cuando se vean excursionistas reales, tengamos un 86% de posibilidades de identificarlos. Cuando se ven árboles o excursionistas, tenemos un 16 % de posibilidades de identificarlos como excursionistas.

Digamos que para nuestra situación particular, consideramos tan importante obtener un verdadero positivo como evitar un falso positivo. No queremos ignorar a los excursionistas en la montaña, pero tampoco queremos enviar a nuestro equipo a condiciones peligrosas sin ningún motivo.

Podemos encontrar el mejor umbral creando nuestro propio sistema de puntuación y viendo qué umbral obtendría el mejor resultado:

In [ ]:
# Calculate how good each threshold is from our TPR and FPR. 
# Our criteria is that the TPR is as high as possible and 
# the FPR is as low as possible. We consider them equally important
scores = thresholds_rf.tpr - thresholds_rf.fpr

# Find the entry with the lowest score according to our criteria
index_of_best_score = numpy.argmax(scores)
best_threshold = thresholds_rf.threshold[index_of_best_score]
print("Best threshold:", best_threshold)

# Print out its expected performance
print("TPR at this threshold:", thresholds_rf.tpr[index_of_best_score])
print("FPR at this threshold:", thresholds_rf.fpr[index_of_best_score])

Nuestro mejor umbral, con este criterio, es 0,74, ¡no 0,5! Esto nos permitiría identificar correctamente al 83 % de los excursionistas, una ligera disminución del 86 %, pero solo identificamos erróneamente al 3,6 % de los árboles como excursionistas.

Si lo desea, juegue con la forma en que calculamos nuestros puntajes aquí y vea cómo se ajusta el umbral.

Resumen
¡Eso es todo! Aquí hemos creado curvas ROC para dos modelos diferentes, utilizando el código que escribimos en el ejercicio anterior.

Visualmente, eran bastante similares, y cuando los comparamos usando la métrica del área bajo la curva, encontramos que el modelo de regresión logística tenía un desempeño marginalmente mejor.

Luego usamos la curva ROC para ajustar nuestro modelo de bosque aleatorio, según criterios específicos de nuestras circunstancias. Nuestros criterios muy simples de TPR - FPR nos permitieron elegir un umbral adecuado para nosotros.